In [39]:
from importlib import reload
import EEG_with_functions as f
reload(f)

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import json

import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import warnings

In [40]:
survey = f.import_survey_df()
features = pd.read_csv("features.csv")

C:\Users\scbui\AppData\Local\Temp\ipykernel_40864\3709109211.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  features = pd.read_csv("features.csv")


0       2
1       3
2       2
3       2
4       3
       ..
2044    3
2045    3
2046    3
2047    2
2048    3
Name: class, Length: 2049, dtype: int64

In [ ]:
%%capture
warnings.filterwarnings("ignore")

res = f.import_data('045_1000619.csv', "Sensor_data_compleet", "./")
info, sampling_freq, n_channels, ch_names, ch_types = f.metadata()
only_electrodes = ch_names
res = f.stim_channels(res)

stimuli_dict, res = f.change_stimuli_names(res)

info, channel_names = f.make_mne_object(info, sampling_freq, n_channels, only_electrodes, ch_types)
res = f.remove_nas(res, only_electrodes)
raw = f.convertto_raw(res, channel_names, info)
raw_filter = f.filter_raw(raw)
events, baseline_list, fixcross_list, stimuli_list = f.extract_events(raw_filter)
epochs = f.create_epochs(raw_filter, events, "no")
low_cutoff, high_cutoff, notch_freq, artifact_threshold, n_fft, n_overlap = f.parameters(1.0, 0.1)


In [ ]:
def create_baseline_waves_df(events, baseline_epochs, only_electrodes, respondent):
    baseline_features = pd.DataFrame()
    res = []
    baseline_list = []
    baseline_ID = []

    for ep in events:
        ep_name = ep[2]
        if str(ep_name).startswith("4"): #baseline LET OP: IS NU ALLEEN FIXCROSSES IPV BASELINE, WANT DE BASELINES ZIJN GENOMEN VANAF DE FIXCROSS
            res.append(respondent)
            baseline_list.append("baseline")
            baseline_ID.append(str(ep_name))

            ep_namestr = str(ep_name)
            spectrum_unique = baseline_epochs[ep_namestr].compute_psd(method ="welch", fmin=4, fmax=20, n_fft= 256, n_overlap= int(0.1 * 256), n_per_seg = 256) #
            
            # baseline_features3["ID"] = ep_namestr
            for ch in only_electrodes:
                for i in range(4, 21):
                    hz = spectrum_unique.get_data(fmin=i, fmax=i, picks = ch)
                    hz_unpacked = hz[0][0][0]
                    column = ch + "_" + str(i) + "hz"
                    baseline_features.loc[ep_namestr, column]  = float(hz_unpacked)
                
    
    baseline_features["Type"] = baseline_list
    baseline_features["ID"] = baseline_ID
    baseline_features["Respondent"] = res
    baseline_features = baseline_features[["Respondent", "ID", "Type"] + [col for col in baseline_features.columns if col not in ["Respondent", "ID", "Type"]]]

    return baseline_features

def create_stimuli_waves_df(events, stimuli_epochs, only_electrodes, respondent):   
    stimuli_features = pd.DataFrame()

    stimuli_str_list_PSD = []
    res = []
    stimuli_ID = []

    for ep in events:
        ep_name = ep[2]
        if str(ep_name).startswith(("5", "6", "7")): #stimuli
            ep_namestr = str(ep_name)
            res.append(respondent)
            stimuli_ID.append(str(ep_name))

            if ep_namestr.startswith("5"):
                ty = "pos"
            elif ep_namestr.startswith("6"):
                ty = "neu"
            elif ep_namestr.startswith("7"):
                ty = "neg"
            stimuli_str_list_PSD.append(ty)

            spectrum_unique = stimuli_epochs[ep_namestr].compute_psd(method ="welch", fmin=4, fmax=20, n_fft= 256, n_overlap= int(0.1 * 256), n_per_seg = 256) #
            for ch in only_electrodes:
                for i in range(4, 21):
                    hz = spectrum_unique.get_data(fmin=i, fmax=i, picks = ch)
                    hz_unpacked = hz[0][0][0]
                    column = ch + "_" + str(i) + "hz"
                    stimuli_features.loc[ep_namestr, column]  = float(hz_unpacked)

    stimuli_features["Type"] = stimuli_str_list_PSD
    stimuli_features["ID"] = stimuli_ID
    stimuli_features["Respondent"] = res
    stimuli_features = stimuli_features[["Respondent", "ID", "Type"] + [col for col in stimuli_features.columns if col not in ["Respondent", "ID", "Type"]]]
    return stimuli_features